# 3. Multi-class Vnet on BV

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from dataset import *
from vnet import *
from training import *
from niiutility import show_image, show_batch_image

%matplotlib inline
%load_ext autoreload
%autoreload 2

## 3.1 Setup Torch Global Variable, load memory map 

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader, sampler, SubsetRandomSampler
from torchvision import transforms, utils

import torch.nn.functional as F  # useful stateless functions
import torchvision.transforms as T

#------------------------------- GLOBAL VARIABLES -------------------------------------#

USE_GPU = True
BATCH_SIZE = 3
NUM_WORKERS = 6
NUM_TRAIN = 72
LEARNING_RATE = 1e-2

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
    print('using GPU for training')
else:
    device = torch.device('cpu')

using GPU for training


In [3]:
#-------------------------LOAD THE DATA SET-------------------------------------------#

data_index = np.arange(107)
data_index = np.delete(data_index, 46)
dataset_trans = BvMaskDataset(data_index, 
                         transform=transforms.Compose([
                             downSample(2),
                             RandomFilp(0.5),
                             RandomAffine(90, 5)
                         ])
                     )

#-------------------------CREATE DATA LOADER FOR TRAIN AND VAL------------------------#

data_size = len(dataset_trans)
train_loader = DataLoader(dataset_trans, batch_size=BATCH_SIZE, \
                    sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)),\
                    num_workers=NUM_WORKERS)
validation_loader = DataLoader(dataset_trans, batch_size=BATCH_SIZE,
                    sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN,data_size)),\
                    num_workers=NUM_WORKERS)

* Print first 4 batch of data

In [4]:
from rpn import RPN3D
#-------------------------NEW MODEL INIT WEIGHT--------------------------------------#

model = RPN3D(img_size=(64, 96, 96), out_size=6)
model.apply(weights_init)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
#optimizer = optim.SGD (model.parameters(),lr=LEARNING_RATE, momentum=0.99)

from loss import *


In [5]:
train(model, train_loader, validation_loader, optimizer,\
      device=device, dtype=dtype, lossFun=IoU, epochs=1500, print_every=12)

epoch 0 begins: 
     Iteration 0, loss = 2.7472
     Iteration 12, loss = 1.9076
     validation loss = 1.0838
epoch 1 begins: 
     Iteration 0, loss = 0.8569
     Iteration 12, loss = 0.8142
     validation loss = 0.9376
epoch 2 begins: 
     Iteration 0, loss = 0.5049
     Iteration 12, loss = 0.5208
     validation loss = 0.5607
epoch 3 begins: 
     Iteration 0, loss = 0.3704
     Iteration 12, loss = 0.3029
     validation loss = 0.4380
epoch 4 begins: 
     Iteration 0, loss = 0.4355
     Iteration 12, loss = 0.5163
     validation loss = 0.5217
epoch 5 begins: 
     Iteration 0, loss = 0.5481
     Iteration 12, loss = 0.4140
     validation loss = 0.3912
epoch 6 begins: 
     Iteration 0, loss = 0.4808
     Iteration 12, loss = 0.4804
     validation loss = 0.4453
epoch 7 begins: 
     Iteration 0, loss = 0.3834
     Iteration 12, loss = 0.5682
     validation loss = 0.4575
epoch 8 begins: 
     Iteration 0, loss = 0.3887
     Iteration 12, loss = 0.5131
     validation loss =

     validation loss = 0.2354
epoch 72 begins: 
     Iteration 0, loss = 0.3778
     Iteration 12, loss = 0.3833
     validation loss = 0.4699
epoch 73 begins: 
     Iteration 0, loss = 0.4922
     Iteration 12, loss = 0.4240
     validation loss = 0.4273
epoch 74 begins: 
     Iteration 0, loss = 0.5348
     Iteration 12, loss = 0.3624
     validation loss = 0.2322
epoch 75 begins: 
     Iteration 0, loss = 0.3690
     Iteration 12, loss = 0.3887
     validation loss = 0.4245
epoch 76 begins: 
     Iteration 0, loss = 0.6038
     Iteration 12, loss = 0.4022
     validation loss = 0.4466
epoch 77 begins: 
     Iteration 0, loss = 0.4207
     Iteration 12, loss = 0.4003
     validation loss = 0.2441
epoch 78 begins: 
     Iteration 0, loss = 0.4352
     Iteration 12, loss = -0.2333
     validation loss = 0.4176
epoch 79 begins: 
     Iteration 0, loss = 0.3909
     Iteration 12, loss = 0.4283
     validation loss = 0.2700
epoch 80 begins: 
     Iteration 0, loss = 0.3662
     Iteration 

Traceback (most recent call last):
  File "/home/xu/anaconda3/envs/cs231/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/home/xu/anaconda3/envs/cs231/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/xu/anaconda3/envs/cs231/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/xu/anaconda3/envs/cs231/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/xu/anaconda3/envs/cs231/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
Traceback (most recent call last):
  File "/home/xu/anaconda3/envs/cs231/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/home/xu/anaconda3/envs/cs231/lib/python3.6/multiprocessing/co

KeyboardInterrupt: 

In [ ]:
#-------------------------SAVE THE MODEL STATE DICT----------------------------------#
PATH = 'Vet_Mask_class_2'
torch.save(model.state_dict(), PATH)